### Installing necessary libraries

In [ ]:
# pip install https://huggingface.co/d4data/en_pipeline/resolve/main/en_pipeline-any-py3-none-any.whl

In [175]:
!pip install datasets Dbias

### import libraries

In [1]:
import evaluate
import pandas as pd

### input and out path of Data set

In [2]:
input_csv_path_eval = "C:/Users/vijaya.sekhar/Desktop/accelarators/Bias_Toxicity/bias_test/detox_test_eval.csv"

input_csv_path_dbias = "C:/Users/vijaya.sekhar/Desktop/accelarators/Bias_Toxicity/bias_test/detox_test_bias.csv"

input_csv_path_classify = "C:/Users/vijaya.sekhar/Desktop/accelarators/Bias_Toxicity/bias_test/detox_test_classify"

output_csv_path = "C:/Users/vijaya.sekhar/Desktop/accelarators/Bias_Toxicity/bias_test/detox_test_output1.csv"

### Bias Detection

In [7]:
# References:
# https://github.com/dreji18/Fairness-in-AI
# https://link.springer.com/article/10.1007/s41060-022-00359-4
import pandas as pd
from Dbias.bias_classification import *
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("d4data/bias-detection-model")
model = TFAutoModelForSequenceClassification.from_pretrained("d4data/bias-detection-model")
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer) # cuda = 0,1 based on gpu availability



def calculate_dbias_detection(input_csv_path, output_csv_path):
    
    # Read the CSV file into a DataFrame
    df = pd.read_csv(input_csv_path)

    # Iterate through columns and calculate row toxicity for each column
    for col_name in df.columns:
        # Create new column names for the toxicity scores and labels
        new_score_col_name = f"{col_name}_bias_score"
        new_label_col_name = f"{col_name}_bias_label"

        # Initialize lists to store scores and labels
        toxicity_scores = []
        toxicity_labels = []

        for text in df[col_name]:
            results = classifier(text,padding=True, truncation=True,max_length=512)
            classify_toxicity = max(results, key=lambda x: x['score'])

            # Append the toxicity score and label
#             toxicity_scores.append(classify_toxicity['score'])
            toxicity_labels.append(classify_toxicity['label'])
            
        # Add the toxicity scores and labels as new columns to the DataFrame
#         df[new_score_col_name] = toxicity_scores
        data_output = pd.read_csv(output_csv_path)
        data_output[new_label_col_name] = toxicity_labels

    # Save the updated DataFrame to a new CSV file
    data_output.to_csv(output_csv_path,index=False)
    print( data_output.head())




Some layers from the model checkpoint at d4data/bias-detection-model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at d4data/bias-detection-model and are newly initialized: ['dropout_79']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Call the function to detect Bias in the sentences
calculate_dbias_detection(input_csv_path_dbias, output_csv_path)

                                           sentences sentences_bias_label
0  I'm having issues with my internet connection,...               Biased
1  Your service is terrible! And Your prices are ...               Biased
2  by playing this game you agree to these terms....           Non-biased
3  you have to use google pokemon trainer club or...           Non-biased
4  don t die or hurt others and if you do it s no...               Biased


### subjectivity

In [9]:
import pandas as pd
from transformers import pipeline

def calculate_classify_subjectivity(input_csv_path, output_csv_path):
    # Initialize the text classification pipeline
    classify = pipeline(task="text-classification", model="cffl/bert-base-styleclassification-subjective-neutral", padding=True, truncation=True,max_length=512, return_all_scores=True)

    # Read the CSV file into a DataFrame
    df = pd.read_csv(input_csv_path)

    # Iterate through columns and calculate row toxicity for each column
    for col_name in df.columns:
        # Create a new column name for the toxicity scores
        new_col_name = f"{col_name}_subjectivity"

        # Calculate toxicity scores and labels for each row in the column
        toxicity_scores = []
        toxicity_labels = []
        for text in df[col_name]:
            results = classify(text)
            classify_toxicity = max(results[0], key=lambda x: x['score'])['label']
            # Find the label with the highest score
#             highest_score_label = max(results, key=lambda x: x['score'])
#             classify_toxicity = highest_score_label['label']
#             classify_score = highest_score_label['score']
#             toxicity_scores.append(classify_score)
            toxicity_labels.append(classify_toxicity)

        # Add the toxicity scores and labels as new columns to the DataFrame
#         df[new_col_name + "_score"] = toxicity_scores
        df[new_col_name + "_label"] = toxicity_labels
        data_output = pd.read_csv(output_csv_path)
        data_output[new_col_name] = toxicity_labels

    # Save the updated DataFrame to a new CSV file
    data_output.to_csv(output_csv_path,index=False)
    print( data_output.head())




In [12]:
# Call the function to calculate subjectivity
calculate_classify_subjectivity(input_csv_path_classify, output_csv_path)

### Toxicity and Emotion

In [248]:
def calculate_evaluate_toxicity(input_csv_path, output_csv_path):
        
    # Read the CSV 
    df = pd.read_csv(input_csv_path)
    # Define toxicity level bins
    bins = [-1, 0.01, 0.1, 0.5, float('inf')]
    labels = ['Not Toxic', 'Less Toxic', 'Moderately Toxic', 'Highly Toxic']
    
    # Iterate through columns for row toxicity for each column ,toxic_label='offensive'
    for col_name in df.columns:
        eval_toxicity = toxicity.compute(predictions=df[col_name].tolist())['toxicity']
        new_col_name = f"{col_name}_evl_toxicity"
        df[new_col_name] = eval_toxicity
        df['toxicity_level'] = pd.cut(df[new_col_name], bins=bins, labels=labels)
    data_output = pd.read_csv(output_csv_path)
    data_output[new_col_name] = eval_toxicity
    data_output[new_col_name] = df['toxicity_level'] 

    # Save the updated DataFrame to a new CSV file
    data_output.to_csv(output_csv_path,index=False)
    print( data_output.head())


In [249]:
calculate_evaluate_toxicity(input_csv_path_eval,output_csv_path)

                                                text  \
0  Unbelievably, with all they had seen their chi...   
1  And while states like Massachusetts�among othe...   
2  Oxford chose "climate emergency" as the word o...   
3  In another clip, the group can be seen scrambl...   
4  Proponents of stricter protections for student...   

  text_dbias_toxicity_label text_cls_toxicity text_evl_toxicity  
0                    Biased        SUBJECTIVE         Not Toxic  
1                    Biased        SUBJECTIVE         Not Toxic  
2                Non-biased           NEUTRAL         Not Toxic  
3                Non-biased           NEUTRAL         Not Toxic  
4                    Biased        SUBJECTIVE         Not Toxic  


In [250]:
import pandas as pd

def calculate_evaluate_emotion(input_csv_path, output_csv_path):
    neutral = ["This is a neutral tweet"]
    df = pd.read_csv(input_csv_path)
    

    for col_name in df.columns:
           
        emotions = []
        new_col_name = f"{col_name}_emotion"
        for text in df[col_name]:
            reg_dict = regard.compute(data=[text], references=neutral)
            reg_dict = pd.DataFrame.from_dict(reg_dict, orient='index').drop(['other'], axis=1).reset_index(drop=True)
            reg_dict['emotion'] = reg_dict.idxmax(axis=1)
#             print(reg_dict['emotion'].values)
            emotions.append(reg_dict['emotion'].values)
        
            
    
    data_output = pd.read_csv(output_csv_path)
    data_output['emotion_type'] = emotions
    print(data_output.head())
    data_output.to_csv(output_csv_path, index=False) 



In [251]:
calculate_evaluate_emotion(input_csv_path_eval,output_csv_path)

                                                text  \
0  Unbelievably, with all they had seen their chi...   
1  And while states like Massachusetts�among othe...   
2  Oxford chose "climate emergency" as the word o...   
3  In another clip, the group can be seen scrambl...   
4  Proponents of stricter protections for student...   

  text_dbias_toxicity_label text_cls_toxicity text_evl_toxicity emotion_type  
0                    Biased        SUBJECTIVE         Not Toxic   [negative]  
1                    Biased        SUBJECTIVE         Not Toxic   [negative]  
2                Non-biased           NEUTRAL         Not Toxic   [negative]  
3                Non-biased           NEUTRAL         Not Toxic   [negative]  
4                    Biased        SUBJECTIVE         Not Toxic   [negative]  


In [3]:
pip install 'langkit[all]'

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: "'langkit[all]'"

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
pip install langkit

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from langkit import llm_metrics # alternatively use 'light_metrics'
import whylogs as why

why.init(session_type='whylabs_anonymous')
# Note: llm_metrics.init() downloads models so this is slow first time.
schema = llm_metrics.init()

[nltk_data] Error loading vader_lexicon: <urlopen error [WinError
[nltk_data]     10060] A connection attempt failed because the
[nltk_data]     connected party did not properly respond after a
[nltk_data]     period of time, or established connection failed
[nltk_data]     because connected host has failed to respond>
